In [64]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/Users/mayureshnene/Desktop/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/Users/mayureshnene/Desktop/indic_nlp_resources"

import sys
import pandas as pd
import numpy as np
import warnings
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from indicnlp import common
from indicnlp import loader
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import indic_tokenize 
from indicnlp.morph import unsupervised_morph 
from indicnlp import common

sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

common.set_resources_path(INDIC_NLP_RESOURCES)

loader.load()


In [65]:
training_dataset = pd.read_csv("/Users/mayureshnene/Desktop/Mayuresh/Offense_Marathi/Data/Data.csv")
training_dataset.dropna()
training_dataset['subtask_c'].fillna("NULL")


0        NULL
1        NULL
2        NULL
3        NULL
4        NULL
         ... 
12990    NULL
12991    NULL
12992    NULL
12993    NULL
12994    NULL
Name: subtask_c, Length: 12995, dtype: object

In [66]:
tweets = training_dataset["tweet"]
tweets

0        आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगाव...
1        कुणी कविता करत असतं तर कुणी कविता जगत असतं कुण...
2        आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आम...
3        गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाष...
4        कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब ...
                               ...                        
12990    पहिल्या लाटे नंतर सर्वात कमी रुग्ण\n#कोरोना #र...
12991    RT @DeshRat30997789: @RavindraAmbekar जगातले स...
12992    Tata Tigor EV | देशातील सर्वात स्वस्त इलेक्ट्र...
12993    RT @PIBMumbai: पंतप्रधान आयुष्मान भारत आरोग्यव...
12994    @Hopeful_Addict काटकसर म्हणजे गोधड्या शिवणे.\n...
Name: tweet, Length: 12995, dtype: object

In [67]:
level_A = training_dataset[["subtask_a"]]
level_B = training_dataset.query("subtask_a == 'Offensive'")[["subtask_b"]]
level_C = training_dataset.query("subtask_b == 'TIN'")[["subtask_c"]]

In [68]:
stopwords_file = open("/Users/mayureshnene/Desktop/MOLD/Mold/stopwords.txt")
stopwords = stopwords_file.read().splitlines()
print(stopwords)

['अधिक ', 'अनेक ', 'अशी ', 'असलयाचे', 'असलेल्या', 'असा', 'असून', 'असे', 'आज', 'आणि', 'आता', 'आपल्या', 'आला', 'आली', 'आले', 'आहे', 'आहेत', 'एक', 'एका', 'कमी', 'करणयात', 'करून', 'का', 'काम', 'काय', 'काही', 'किवा', 'की', 'केला', 'केली', 'केले', 'कोटी', 'गेल्या', 'घेऊन', 'जात', 'झाला', 'झाली', 'झाले', 'झालेल्या', 'टा', 'डॉ', 'तर', 'तरी', 'तसेच', 'ता', 'ती', 'तीन', 'ते', 'तो', 'त्या', 'त्याचा', 'त्याची', 'त्याच्या', 'त्याना', 'त्यानी', 'त्यामुळे', 'त्री', 'दिली', 'दोन', 'न', 'नाही', 'निर्ण्य', 'पण', 'पम', 'परयतन', 'पाटील', 'म', 'मात्र', 'माहिती', 'मी', 'मुबी', 'म्हणजे', 'म्हणाले', 'म्हणून', 'या', 'याचा', 'याची', 'याच्या', 'याना', 'यानी', 'येणार', 'येत', 'येथील', 'येथे', 'लाख', 'व', 'व्यकत', 'सर्व', 'सागित्ले', 'सुरू', 'हजार', 'हा', 'ही', 'हे', 'होणार', 'होत', 'होता', 'होती', 'होते']


In [69]:
def clean(row):
    row = str(row)
#     removal_list = ['URL','\'ve','n\'t','\'s','\'m','!']
#     for element in removal_list:
#         row = row.replace(element,'')
    
    row = row.replace('http\S+|www.\S+', '')
    row = re.sub("@[A-Za-z0-9]+","@USER",row)
    row = re.sub("[A-Za-z0-9]+","",row)
    row = re.sub("@","@USER",row)
    row = re.sub('[+,-,_,=,/,<,>,!,#,$,%,^,&,*,\",:,;,.,' ',\t,\r,\n,\',|]','',row)
    return row

In [70]:
# for tweet in tweets:
#     tweet = remove_noise(str(tweet))
iterator_map = map(clean,tweets)
tweets = list(iterator_map)

In [71]:
  

collective_tweets = copy.deepcopy(training_dataset)

In [72]:


analyzer=unsupervised_morph.UnsupervisedMorphAnalyzer('mr')


In [73]:
def tokenize(tweet):
    return indic_tokenize.trivial_tokenize(tweet)

def morph(tweet):
    analyzed_tokens=analyzer.morph_analyze_document(str(tweet).split(' '))
    return analyzed_tokens

In [74]:
tqdm.pandas(desc="Tokenize..")
#all_tweets["tokens"] = all_tweets['tweet'].progress_apply(tokenize)
collective_tweets["tokens"] = collective_tweets['tweet'].progress_apply(morph)

Tokenize..: 100%|██████████| 12995/12995 [00:02<00:00, 4762.07it/s]


In [75]:
vector = collective_tweets["tokens"].tolist()

In [76]:
def tfid_vectorizer(vector):
	## Creates and stores an instance of the TfidfVectorizer class. This will be used further to extract our data as tf-idf features.
	vectorizer = TfidfVectorizer()
	untokenized_data =[' '.join(tweet) for tweet in tqdm(vector, "Vectorizing...")]
	vectorizer = vectorizer.fit(untokenized_data)
	vectors = vectorizer.transform(untokenized_data).toarray()
	return vectors

def get_vectors(vectors, labels, keyword):
	'''
	Returns a matrix for vectors. Zips vectors and labels IF and only if length of vector list is the same as length of the labels list. 
	Else, the function gets terminated.
	@param vectors These are the vectors for a given label.
	@param labels These are the label values for the given label.
	@param keyword which is the label to annotate for.
	'''
	if len(vectors) != len(labels):
		print("Unmatching sizes!")
		return
	
	## Stores a new list to append the zipped vectors and labels into.
	result = list()
	for vector, label in zip(vectors, labels):
		if label == keyword:
			result.append(vector)
	return result

In [77]:
vectors_level_A = tfid_vectorizer(vector)
labels_level_a = level_A['subtask_a'].values.tolist()
vectors_level_B = get_vectors(vectors_level_A, labels_level_a, "Offensive") 

labels_level_b = level_B['subtask_b'].values.tolist() 

## Numerical Vectors C
vectors_level_c = get_vectors(vectors_level_B, labels_level_b, "TIN") 

##Subtask C Labels
labels_level_c = level_C['subtask_c'].values.tolist() 

Vectorizing...: 100%|██████████| 12995/12995 [00:00<00:00, 793677.09it/s]


In [ ]:
train_vectors_level_A, train_labels_level_A,= vectors_level_A[1:], labels_level_a[1:]
test_vectors_level_A, test_labels_level_A = vectors_level_A[3097:], labels_level_a[3097:]
## Extracting names of labels and storing them in a variable
classNames = np.unique(test_labels_level_A)

print("Training begins on Level A classification...")
warnings.filterwarnings(action='ignore')

## Creating an object of SVC
classifiersvc = SVC()
classifiermnb = MultinomialNB()
classifiersgd = SGDClassifier()
classifiermlp = MLPClassifier()

classifiersvc.fit(train_vectors_level_A, train_labels_level_A)
classifiermnb.fit(train_vectors_level_A, train_labels_level_A)
classifiersgd.fit(train_vectors_level_A, train_labels_level_A)
classifiermlp.fit(train_vectors_level_A, train_labels_level_A)

# accuracy = accuracy_score(train_labels_level_A, classifiersvc.predict(train_vectors_level_A))
# print("Training Accuracy:", accuracy)


test_predictions_A_svc = classifiersvc.predict(test_vectors_level_A)
test_predictions_A_mnb = classifiermnb.predict(test_vectors_level_A)
test_predictions_A_sgd = classifiersgd.predict(test_vectors_level_A)
test_predictions_A_mlp = classifiermlp.predict(test_vectors_level_A)


accuracy_svc = accuracy_score(test_labels_level_A, test_predictions_A_svc)
accuracy_mnb = accuracy_score(test_labels_level_A, test_predictions_A_mnb)
accuracy_sgd = accuracy_score(test_labels_level_A, test_predictions_A_sgd)
accuracy_mlp = accuracy_score(test_labels_level_A, test_predictions_A_mlp)
print("Level A: \n")
print("Test Accuracy SVC:", accuracy_svc)
print("Test Accuracy MNB:", accuracy_mnb)
print("Test Accuracy SGD:", accuracy_sgd)
print("Test Accuracy MLP:", accuracy_mlp)




preds_A = pd.DataFrame(columns = ['SVC_A', 'MNB_A', 'SGD_A','MLP_A'])
label_name = ['SVC_A', 'MNB_A','SGD_A','MLP_A']
preds = [test_predictions_A_svc, test_predictions_A_mnb, test_predictions_A_sgd, test_predictions_A_mlp]

i = 0
for label in label_name:
    preds_A[label] = preds[i]
    i += 1



Training begins on Level A classification...


In [ ]:
## Split into Train and Test vectors using the vectors of level A and Labels of level B with a training size of 0.75.
train_vectors_level_B, test_vectors_level_B, train_labels_level_B, test_labels_level_B = train_test_split(vectors_level_B[:], labels_level_b[:], train_size=0.75)

## Extracting names of labels and storing them in a variable
classNames = np.unique(test_labels_level_B)
print("Training begins on Level B classification...")
warnings.filterwarnings(action='ignore')

## Creating an object of SVC
classifiersvc = SVC()
classifiermnb = MultinomialNB()
classifiersgd = SGDClassifier()
classifiermlp = MLPClassifier()

classifiersvc.fit(train_vectors_level_B, train_labels_level_B)
classifiermnb.fit(train_vectors_level_B, train_labels_level_B)
classifiersgd.fit(train_vectors_level_B, train_labels_level_B)
classifiermlp.fit(train_vectors_level_B, train_labels_level_B)

print("Training complete....")


print("calculating accuracy....")
## Training accuracy has been calculated
accuracy = accuracy_score(train_labels_level_B, classifiersvc.predict(train_vectors_level_B))
print("Training Accuracy:", accuracy)

## predictions are obtained on the testing data set
test_predictions_B_svc = classifiersvc.predict(test_vectors_level_B)
test_predictions_B_mnb = classifiermnb.predict(test_vectors_level_B)
test_predictions_B_sgd = classifiersgd.predict(test_vectors_level_B)
test_predictions_B_mlp = classifiermlp.predict(test_vectors_level_B)

## Testing accuracy has been calculated
accuracy_svc = accuracy_score(test_labels_level_B, test_predictions_B_svc)
accuracy_mnb = accuracy_score(test_labels_level_B, test_predictions_B_mnb)
accuracy_sgd = accuracy_score(test_labels_level_B, test_predictions_B_sgd)
accuracy_mlp = accuracy_score(test_labels_level_B, test_predictions_B_mlp)
print("Level B: \n")
print("Test Accuracy SVC:", accuracy_svc)
print("Test Accuracy MNB:", accuracy_mnb)
print("Test Accuracy SGD:", accuracy_sgd)
print("Test Accuracy MLP:", accuracy_mlp)

print("Confusion Matrix:")
## confusion matrix has been obtained for level A classification
matrix_level_B = confusion_matrix(test_labels_level_B, test_predictionsB)
print(matrix_level_B)
## Obtaining classification report for the test data set
print(classification_report(test_labels_level_B,test_predictionsB))

# ## Plotting confusion matrix for better visualization
# plottedCM_Level_B = plot_confusion_matrix(classifiersvc, test_vectors_level_B, test_labels_level_B, display_labels=classNames, cmap=plt.cm.Blues)
# plt.show()





## Split into Train and Test vectors using the vectors of level A and Labels of level B with a training size of 0.75.
train_vectors_level_C, test_vectors_level_C, train_labels_level_C, test_labels_level_C = train_test_split(vectors_level_c[:], labels_level_c[:], train_size=0.75)

## Extracting names of labels and storing them in a variable
classNames = np.unique(test_labels_level_C)
print("Training begins on Level C classification...")
warnings.filterwarnings(action='ignore')

## Creating an object of SVC
classifiersvc = SVC()
classifiermnb = MultinomialNB()
classifiersgd = SGDClassifier()
classifiermlp = MLPClassifier()

classifiersvc.fit(train_vectors_level_C, train_labels_level_C)
classifiermnb.fit(train_vectors_level_C, train_labels_level_C)
classifiersgd.fit(train_vectors_level_C, train_labels_level_C)
classifiermlp.fit(train_vectors_level_C, train_labels_level_C)



## predictions are obtained on the testing data set
test_predictions_C_svc = classifiersvc.predict(test_vectors_level_C)
test_predictions_C_mnb = classifiermnb.predict(test_vectors_level_C)
test_predictions_C_sgd = classifiersgd.predict(test_vectors_level_C)
test_predictions_C_mlp = classifiermlp.predict(test_vectors_level_C)


accuracy_svc = accuracy_score(test_labels_level_C, test_predictions_C_svc)
accuracy_mnb = accuracy_score(test_labels_level_C, test_predictions_C_mnb)
accuracy_sgd = accuracy_score(test_labels_level_C, test_predictions_C_sgd)
accuracy_mlp = accuracy_score(test_labels_level_C, test_predictions_C_mlp)
print("Level C: \n")
print("Test Accuracy SVC:", accuracy_svc)
print("Test Accuracy MNB:", accuracy_mnb)
print("Test Accuracy SGD:", accuracy_sgd)
print("Test Accuracy MLP:", accuracy_mlp)

In [ ]:
preds_B = pd.DataFrame(columns = ['SVC_B', 'MNB_B', 'SGD_B','MLP_B'])
label_name = ['SVC_B', 'MNB_B','SGD_B','MLP_B']
preds = [test_predictions_B_svc, test_predictions_B_mnb, test_predictions_B_sgd, test_predictions_B_mlp]

i = 0
for label in label_name:
    preds_B[label] = preds[i]
    i += 1

In [ ]:
preds_C = pd.DataFrame(columns = ['SVC_C', 'MNB_C', 'SGD_C','MLP_C'])
label_name = ['SVC_C', 'MNB_C','SGD_C','MLP_C']
preds = [test_predictions_C_svc, test_predictions_C_mnb, test_predictions_C_sgd, test_predictions_C_mlp]

i = 0
for label in label_name:
    preds_C[label] = preds[i]
    i += 1


In [ ]:
#final_df = pd.DataFrame()
final_df = pd.concat([preds_A, preds_B, preds_C], ignore_index=True, sort=False)

In [ ]:
final_df

In [ ]:
final_df.to_csv("/Users/mayureshnene/Desktop/Mayuresh/Offense_Marathi/Experiments/Data_annotated.csv")